In [2]:
# 한컴오피스 자동화 입문1 : 상장 자동제작 feat.파이썬
# https://youtu.be/1pC_UXhiJH8

import win32com.client as win32
import pandas as pd

In [3]:
# hwp = win32.Dispatch("HWPFrame.HwpObject")		# 만약 오류가 발생한다면 : pywintypes.com_error: (-2147352562, '매개 변수의 개수가 잘못되었습니다.', None, None)
hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")

# 보안모듈 등록 : 오류난다면 레지스트리 키의 철자, 경로 확인할 것
hwp.RegisterModule('FilePathCheckDLL', "SecurityModule")

True

In [9]:
# 모든 시트 불러오기 : sheet_name=None
# 헤더(= 컬럼 니름 = 1행) 없으면 : header=None
# 인덱스 만들고 싶지 않으면 : index_col=None
excel = pd.read_excel("상속 이전 안내문 발송대상자 명단(5월~8월).xlsx", sheet_name=None)

excel

{'대상자명단(5월)':    순번         수신일       사망신고일  자동차등록번호          주민등록번호   성명   소유구분  \
 0   1  2021.05.04  2021.05.03  02버2585  450818-1068514  신병호  대표소유자   
 1   2  2021.05.05  2021.05.04  11구1488  380311-1011123  김동기  대표소유자   
 2   3  2021.05.11  2021.05.10  51러0795  330506-2042217  이규남  대표소유자   
 3   4         NaN         NaN  51러0795  641228-2042311  김민정  공동소유자   
 4   5  2021.05.14  2021.05.13  07노5689  541011-1341825  이승기  대표소유자   
 5   6  2021.05.18  2021.05.17  32조4075  650419-1009414  김오남  대표소유자   
 6   7  2021.05.25  2021.05.24  03거8445  520705-1221211  정태희  대표소유자   
 7   8  2021.05.26  2021.05.25  02루0254  600319-1051121  최재유  공동소유자   
 8   9         NaN         NaN  02루0254  960105-1064921   최호  대표소유자   
 9  10  2021.05.26  2021.05.25  06마3459  350322-1038015  이강복  대표소유자   
 
                                                주민주소  주민주소 우편번호  \
 0          서울특별시 노원구 마들로 31, 105동 204호(월계동, 그랑빌아파트)       1913   
 1           서울특별시 노원구 노원로 58, 405동 804호(공릉동, 우방아파트)       1799   
 2 

In [10]:
# 시트 이름 확인
excel.keys()

dict_keys(['대상자명단(5월)', '대상자명단(6월)', '대상자명단(7월)', '대상자명단(8월)'])

In [12]:
# 특정 시트만 불러오기 : excel['대상자명단(5월)']
excel['대상자명단(5월)']

In [13]:
# 전체 시트를 하나로 합치기(인덱스 무시)
excel_total = pd.concat(excel, ignore_index=True)

excel_total

In [26]:
# 특정 열만 추출하기
excel_regnum_name = excel_total[['자동차등록번호', '성명']]
excel_regnum_name

,자동차등록번호,성명
0,02버2585,신병호
1,11구1488,김동기
2,51러0795,이규남
3,51러0795,김민정
4,07노5689,이승기
...,...,...
66,47모1438,김영덕
67,67노5582,김제원
68,67노5582,박용식
69,80루3704,길성희


In [27]:
# 열이름 바꾸기
excel_regnum_name.columns=['등록번호', '소유자']
excel_regnum_name

,등록번호,소유자
0,02버2585,신병호
1,11구1488,김동기
2,51러0795,이규남
3,51러0795,김민정
4,07노5689,이승기
...,...,...
66,47모1438,김영덕
67,67노5582,김제원
68,67노5582,박용식
69,80루3704,길성희


In [28]:
#hwp.Open("C:/HWP_API/award.hwp")		# 만약 오류가 발생한다면 : pywintypes.com_error: (-2147352562, '매개 변수의 개수가 잘못되었습니다.', None, None)
hwp.Open("C:/HWP_API/상속 이전 등록 안내문.hwp", "HWP", None)

True

In [29]:
hwp.GetFieldList()

'등록번호\x02소유자'

In [30]:
hwp.GetFieldList(1)

'등록번호{{0}}\x02소유자{{0}}'

In [31]:
hwp.GetFieldList(2)

'등록번호{{1}}\x02소유자{{1}}'

In [32]:
field_list = [i for i in hwp.GetFieldList().split('\x02')]

In [33]:
field_list

['등록번호', '소유자']

In [34]:
# 총 몇명인지 확인
len(excel_regnum_name)

71

In [35]:
# pandas 복습
# excel["열이름"]  ==> DataFrame의 특정 열

excel_regnum_name["등록번호"]

0     02버2585
1     11구1488
2     51러0795
3     51러0795
4     07노5689
       ...   
66    47모1438
67    67노5582
68    67노5582
69    80루3704
70    27고7834
Name: 등록번호, Length: 71, dtype: object

In [38]:
# pandas 복습
# excel["열이름"].iloc[row인덱스]  ==> DataFrame의 특정 열 & 특정 행

excel_regnum_name["등록번호"].iloc[0:3]

0    02버2585
1    11구1488
2    51러0795
Name: 등록번호, dtype: object

In [114]:
# DataFrame에서 값 읽어오기
# 날짜 포맷 : 2021-10-10 00:00:00

for row in range(len(excel)):
    for field in field_list:
        print(excel[field].iloc[row], end='\t')
    print()

1	대상	박혜경	박혜경	2021년 1월 10일	
2	최우수상	이보람	이보람	2021년 2월 1일	
3	금상	김남우	김남우	2021년 3월 8일	
4	은상	조남일	조남일	2021년 8월 1일	
5	동상	최고집	최고집	2021년 10월 14일	
6	장려상	수한무	수한무	2021년 10월 15일	


In [70]:
# 엑셀의 날짜 문자열이 timestamp로 입력된다.
# 이때 시간부분을 잘라내려면 timestamp.date()
# 반환값의 type은 datetime.date

# for row in range(len(excel)):
#     excel['시상일'].iloc[row] = excel['시상일'].iloc[row].date()


C:\ProgramData\Anaconda3\envs\pr\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [72]:
# type(excel['시상일'].iloc[0])

datetime.date

In [113]:
# 엑셀의 날짜 문자열이 timestamp로 입력된 경우
# 'yyyy-mm-dd' 형식이 아니라 'yyyy년 mm월 dd일' 형식으로 맞추려면
# 아래와 같이 포맷한다.
for row in range(len(excel)):
    excel['시상일'].iloc[row] = f"{excel['시상일'].iloc[row].year}년 {excel['시상일'].iloc[row].month}월 {excel['시상일'].iloc[row].day}일"

C:\ProgramData\Anaconda3\envs\pr\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [117]:
hwp.MovePos(0)    # 가장 처음부분으로

True

In [119]:
hwp.Run('SelectAll')

In [120]:
hwp.Run('Copy')    # hwp.Run('Cut')    hwp.Run('Paste')

In [121]:
hwp.MovePos(3)

True

In [122]:
# 1페이지의 와꾸를 상장 수상자 수만큼 복사한다.
# 엑셀 파일에 총 6명의 수상자
# 따라서 "전체선택"+"복사" 후에 5번(0~4)을 "붙여넣기"해준다

for i in range(len(excel)-1):
    hwp.Run('Paste')
    hwp.MovePos(3)    # 문서 맨끝으로=3

hwp.MovePos(3)

In [123]:
import time
# 1~6페이지(index=0~5)까지 와꾸 완성됐으면
# 이제 페이지마다(0~5) 누름틀 필드에 값을 집어넣음

for page in range(len(excel)):    # 모든 페이지에
    for field in field_list:    # 각 페이지의 모든 필드(누름틀)
        hwp.MoveToField(f"{field}{{{{{page}}}}}")  # 커서를 해당 누름틀로 이동
        time.sleep(1)
        hwp.PutFieldText(f"{field}{{{{{page}}}}}", excel[field].iloc[page])